In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os
import scipy
import sqlite3
from collections import defaultdict

In [3]:
curr_path = os.path.realpath('')
print(curr_path)
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
cur = conn.cursor()
cur.execute("select * from loan limit 5;")
results = cur.fetchall()
# print(results)
cur.close()
conn.close()

C:\Users\Andrew Scott Evans\Documents\GitHub\loan_driving2


In [4]:
conn = sqlite3.connect("lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status = 'Charged Off';", conn)
# print(df)
conn.close()

In [5]:
ddict = df.to_dict()
# print(ddict['loan_amnt'][7])
ddict['sub_grade'][7]
g = zip(ddict['loan_amnt'].values(),ddict['sub_grade'].values(),ddict['id'].values(),ddict['loan_status'].values())
# print(len(list(g)))

In [6]:
# create a training set
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)';", conn)
# print(df.axes)
# print(df.get_values()[0][1])
# print(df.get_values())
# print(df.values[3])
# print(df.values[0])

conn.close()
# feature vector - predictor will predict ("charged off/default status") vs. 
# [1(maybe check this),'sub_grade','loan_amnt','emp_length','home_ownership','annual_inc','verification_status','dti'(debt-to-income),'']

In [7]:
# find all unique values for the loan status column
l_status_dict = defaultdict(int)
for row in df.itertuples():
    # print(row[3])
    if row[3] not in l_status_dict:
        l_status_dict[row[3]] += 1

# print(l_status_dict.keys())

In [8]:
print(l_status_dict.keys())

dict_keys(['Fully Paid', 'Charged Off', 'Default', 'In Grace Period', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off', 'Issued'])


In [9]:
curr_path = os.path.realpath('')
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
# very important, select a random sample of the data for our training set & validation sets 
# 
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 200000);", conn)
df_default = pd.read_sql_query( "select * from loan where loan_status='Charged Off' OR loan_status='Default' OR loan_status='Late (31-120 days)' OR loan_status='Late (16-30 days)' OR loan_status='Does not meet the credit policy. Status:Charged Off' AND id in(select id from loan order by random());", conn)
df_fully_paid = pd.read_sql_query("select * from loan where loan_status='Issued' OR loan_status='Fully Paid' OR loan_status='In Grace Period' OR loan_status='Does not meet the credit policy. Status:Fully Paid' AND id in(select id from loan order by random());" , conn)
# df_default = pd.read_sql_query("select",conn)
conn.close()
# df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 200000);", conn)
# df_validation = pd.read_sql_query("select * from loan where id in(selec id from loan order by random() limit 100000);", conn)
# print(df)

In [10]:
# split the data sets into training and validation
#for row in df.itertuples():
#   print(row[1])
# shuffle that dataframe frac = 1 means return all elements sshuffled not a preportion
df = df.sample(frac=1).reset_index(drop=True)
# print(df)
# grab a random set of 25,000 "default" examples
df_default_pick = df_default.sample(frac=(25000/(float(len(df_default))))).reset_index(drop=True)
# grab a random set of 25,000 "fully paid" examples
df_fully_paid_pick = df_fully_paid.sample(frac=(25000/(float(len(df_fully_paid))))).reset_index(drop=True)

# data_train = df.iloc[:int(len(df)/2), :]
# append the training sets together first 25,000 defaults, next 25,000 non-defaults
data_train = df_default_pick.append(df_fully_paid_pick)
# data_validation = df.iloc[int(len(df)/2):, :]
print(len(df_default))
print(len(df_fully_paid))
print(len(data_train))
print(data_train)


61176
224425
50000
        index        id   member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0      325605  44876424  47954195.0    20000.0      20000.0     20000.000000   
1       37574    379465    405903.0    14000.0      14000.0     13896.090645   
2       54635  30175683  32718901.0    11100.0      11100.0     11100.000000   
3      186008   1106937   1343487.0    12000.0      12000.0     12000.000000   
4      394055  38610029  41393959.0    10000.0      10000.0     10000.000000   
5      175805   1266664   1510199.0    13475.0      13475.0     13450.000000   
6      193792  12956975  14989161.0    30000.0      30000.0     30000.000000   
7       19499   8614925  10387050.0     5475.0       5475.0      5475.000000   
8       35855   7730393   9431814.0     8500.0       8500.0      8500.000000   
9       63695  29193517  31726680.0     5000.0       5000.0      5000.000000   
10      43835   7444706   9106857.0     4000.0       4000.0      4000.000000   
11       7455    8823

In [11]:
# set up 

sub_g_labels = ['A1','A2','A3','A4','A5','B1','B2','B3','B4','B5','C1','C2','C3','C4','C5','D1','D2','D3'\
            ,'D4','D5','E1','E2','E3','E4','E5','F1','F2','F3','F4','F5','G1','G2','G3','G4','G5']

# make sub_grade 
# give sub grade numeric value.
grade_v = [(1.0/35 * i) for i in range(1, 36)]

# dictionary to make sub_grade val to sub_grade string
# sub_grade_dict = dict(zip(sub_g_labels, grade_v))
sub_grade_dict = dict(zip(sub_g_labels, range(1, 36)))

print(sub_grade_dict)
print(df.axes)

{'A1': 1, 'A2': 2, 'A3': 3, 'A4': 4, 'A5': 5, 'B1': 6, 'B2': 7, 'B3': 8, 'B4': 9, 'B5': 10, 'C1': 11, 'C2': 12, 'C3': 13, 'C4': 14, 'C5': 15, 'D1': 16, 'D2': 17, 'D3': 18, 'D4': 19, 'D5': 20, 'E1': 21, 'E2': 22, 'E3': 23, 'E4': 24, 'E5': 25, 'F1': 26, 'F2': 27, 'F3': 28, 'F4': 29, 'F5': 30, 'G1': 31, 'G2': 32, 'G3': 33, 'G4': 34, 'G5': 35}
[RangeIndex(start=0, stop=271653, step=1), Index(['loan_amnt', 'id', 'loan_status', 'sub_grade'], dtype='object')]


In [14]:
# need to build my feature vecotr
# X_train = f['sub_grade','loan_amnt']
# need y predictions combine(charged off/default(True(1)) or fully paid(False(0)))
# y_train =[]
# 11th spot in the pandas df if read in all columns
sub_grade = 11
loan_amnt = 4
loan_status = 18

def feature(datum):
    # create our simplistic featur vector
    #print(datum)
    s_grade = datum[sub_grade]
    # print(datum[sub_grade])
    return [sub_grade_dict[datum[sub_grade]], datum[loan_amnt]]


default = 0
paid = 0
def prediction_of_y(datum):
    if ((datum[loan_status] == 'Charged Off') or (datum[loan_status] == 'Default') or (datum[loan_status] == 'Late (31-120 days)' ) or (datum[loan_status] == 'Late (16-30 days)') or (datum[loan_status] =='Does not meet the credit policy. Status:Charged Off')):
        global default 
        default += 1
        return True
    else:
        global paid 
        paid += 1
        return False

# X_train = [feature(d) for d in data_train]
X_train = []
for row in data_train.itertuples():
    X_train.append(feature(row))
    
# y_train = [prediction_of_y(d) for d in data_train]
y_train = []
paid = 0
for row in data_train.itertuples():
    y_train.append(prediction_of_y(row))
    

In [15]:
l_status_dict = defaultdict(int)
for row in df.itertuples():
    # print(row[3])
    if row[loan_status] not in l_status_dict:
        l_status_dict[row[loan_status]] += 1
        
print(l_status_dict.keys())
print(default)
print(paid)

IndexError: tuple index out of range

In [16]:
# a = list(zip(X_train[:50],y_train[:50]))
# print(a)
# pipline to scale the feature vectors and run the svm
rbf_svm_clf = Pipeline((
        ('scaler', StandardScaler()),
        ("rbf_svc", SVC(kernel='rbf', C=1))
    ))

# scaler.transform(X_train)

In [17]:
# clf = svm.SVC(C=1, kernel='rbf')
# clf.fit(scaler.transform(X_train), y_train)
rbf_svm_clf.fit(X_train, y_train)
train_predictions = rbf_svm_clf.predict(X_train)
# test_predictions = clf.predict(X_test)
print(train_predictions)

[ True  True  True ..., False False False]


In [18]:
print(sum(train_predictions))

25530


In [19]:
numFalseNeeded = 1 - sum(y_train)/len(y_train)
print(sum(y_train))
print(numFalseNeeded)

25000
0.5


In [20]:
# what % acc did we get?

results = list(zip(train_predictions,y_train))
correct = 0
for i in range(len(results)):
    if results[i][0] == results[i][1]:
        correct +=1

print(correct/float(len(results)))

0.62756
